In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-19"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1102,2024-06-19,Mundo Amistoso Internacional,13:00,Alemanha U20,Grécia U20,1.49,2.50,140.5,1.85,1.85,0.0,0.00,0.00,8vus4VfL,0.671141,0.400000,0.540541,0.540541,0.071141,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,86,74,1.28,1.12,0.000,0.000,0.357984,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1103,2024-06-19,Mundo Amistosos Internacionais,13:00,Republica Checa F,Finlândia F,1.22,3.98,136.5,1.87,1.83,0.0,0.00,0.00,4QQRQtH6,0.819672,0.251256,0.534759,0.546448,0.070928,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,65,0,4.13,0.00,0.000,0.000,0.750621,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1104,2024-06-19,Nova Zelândia Nbl,04:30,Auckland Tuatara,Otago Nuggets,1.40,2.80,181.5,1.83,1.87,0.0,0.00,0.00,4Ci2I0in,0.714286,0.357143,0.546448,0.534759,0.071429,114.096,21.610758,0.189409,3.0,0.000000,0.000000,104.00,1.140,0.174786,0.153321,100.0,348.976,289.667749,0.830051,1.2,1.643168,1.369306,470.80,3.730,3.100734,0.831296,-8.0,100,88,1.04,5.35,120.364,271.982,0.471405,0.015289,NaN,0.66,0.132,3.030303,0.000000,0.0,0.000000,-3.55,-0.710,-2.535211,0.000000,0.0,0.000000
1105,2024-06-19,Venezuela Superliga,20:00,Gaiteros del Zulia,Panteras,1.57,2.31,151.5,1.81,1.89,0.0,0.00,0.00,06PPqBPM,0.636943,0.432900,0.552486,0.529101,0.069843,178.952,68.436330,0.382428,1.2,1.643168,1.369306,128.16,2.584,1.229260,0.475720,-5.0,220.038,60.368513,0.274355,1.2,1.643168,1.369306,310.44,2.962,0.763459,0.257751,-16.0,89,78,1.44,3.98,176.888,252.402,0.269721,0.030578,NaN,2.09,0.418,1.363636,0.000000,0.0,0.000000,1.92,0.384,3.411458,0.000000,0.0,0.000000
1106,2024-06-19,Mundo Amistosos Internacionais,13:00,Grécia F,Dinamarca F,1.07,7.38,143.5,1.85,1.85,0.0,0.00,0.00,ImB1BMBU,0.934579,0.135501,0.540541,0.540541,0.070081,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,87,0,1.40,0.00,0.000,0.000,1.056058,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1107,2024-06-19,Canadá Cebl,12:00,Niagara River,Scarborough Shooting Stars,1.56,2.21,178.5,1.80,1.90,0.0,0.00,0.00,Yia7dvF6,0.641026,0.452489,0.555556,0.526316,0.093514,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,86,86,1.54,1.54,0.000,0.000,0.243830,0.038222,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1108,2024-06-19,Canadá Cebl,20:30,Brampton Honey Badgers,Montreal Alliance,1.78,1.89,171.5,1.85,1.85,1.0,1.81,1.83,SEbBebUC,0.561798,0.529101,0.540541,0.540541,0.090898,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,76,93,2.17,2.37,0.000,0.000,0.042388,0.000000,0.007770,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1109,2024-06-19,Mundo Amistoso Internacional,10:30,Bélgica U20

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
1105,20:00,Venezuela Superliga,Gaiteros del Zulia,Panteras,1.57,Back Home
1122,08:00,Indonésia Ibl,Rajawali Medan,Bumi Borneo,2.53,Back Home
